**Import Libraries**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


**Read and Prepaired the Dataset**

In [2]:
# Read the  dataset
data = pd.read_csv('spam.csv', encoding='latin1')
data = data[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})

#label Encodeing: ham -> 0, spam -> 1
data['label'] = data['label'].map({'ham': 0, 'spam': 1})


**Split the Train and Test Dataset**

In [3]:

X = data['message']
y = data['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Transform Text Data using TF-IDF**

In [4]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)


X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

**Train a Model**
**I'm using Logistic Regression**

In [5]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

**Evaluate the Model**

In [6]:
y_pred = model.predict(X_test_tfidf)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.97      0.67      0.79       150

    accuracy                           0.95      1115
   macro avg       0.96      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115

Accuracy: 0.9524663677130045


**Save the Model**

In [7]:
import pickle

with open('spam_classifier_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)


**Test the Model**

In [8]:
with open('spam_classifier_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

#make predictions
new_messages = ["Congratulations! You've won a $1,000 Walmart gift card.",
                "Hey, are we still on for dinner tonight?"]
new_messages_tfidf = loaded_vectorizer.transform(new_messages)
predictions = loaded_model.predict(new_messages_tfidf)

# Interpret predictions
for msg, pred in zip(new_messages, predictions):
    print(f"Message: '{msg}' => {'Spam' if pred == 1 else 'Ham'}")


Message: 'Congratulations! You've won a $1,000 Walmart gift card.' => Ham
Message: 'Hey, are we still on for dinner tonight?' => Ham
